In [ ]:
from pathlib import Path
import pandas as pd
# import numpy as np
import polars as pl
# import duckdb
import os
import pandera as pa
# import io

pd.set_option('display.max_columns', 35)
pd.set_option("display.max_colwidth",200)
pd.set_option("display.max_rows", 500)

FROM_EXPERIMENT_CSV_IN = Path(r"E:\app_data\dropbox_13f_files\processed_tables\TR_02_EXP_SELECT_CIK_CSV")
TO_EXPERIMENT_PARQUET = Path(r"E:\app_data\dropbox_13f_files\processed_tables\TR_03_EXP_SELECT_CIK_PARQUET")

REDUCED_SELECT_CSV =Path(r"E:\app_data\dropbox_13f_files\processed_tables\TR_02_EXP_SELECT_CIK_CSV_REDUCED")

#----
processed_tables_copy = Path(r"E:\app_data\dropbox_13f_files\processed_tables\processed_tables_copy")
TR_00_ALL_CSV_REDUCED = Path(r"E:\app_data\dropbox_13f_files\processed_tables\TR_00_ALL_CSV_REDUCED")

### Transforming original `csv` files into new `csv` with reduced number of columns
* new folder with reduced `csv`: 
  1. From `processed_tables_copy =Path(r"E:\app_data\dropbox_13f_files\processed_tables\processed_tables_copy")`
  2. To ```TR_00_ALL_CSV_REDUCED = Path(r"E:\app_data\dropbox_13f_files\processed_tables\TR_00_ALL_CSV_REDUCED")```
  
* It's a one-off code


In [ ]:
%%time

columns = ['cik', 'cusip8', 'cusip9','value', 'shares','rdate', 'fdate',\
           'address', 'form', 'shrsOrPrnAmt', 'putCall', 'nameOfIssuer', 'titleOfClass', 'type', 'dsource']

dtypes = {'cusip8': str, 'cusip9': str , 'titleOfClass': str, 'form': str,
          'putCall': str, 'shrsOrPrnAmt': str, 'value': pl.Float64, 'shares': pl.Float64, 
          'nameOfIssuer': str, 'cik' : pl.Int64, 'address': str, 'type': str,'num5': str,
          'deviation':str, 'shrout':str,'num3': str,'num2': str, 'num6':str,'num7': str,'num4': str,
           'votingAuthority': str, 'in_universe': str,'prc': str, 'split': str,
           'investmentDiscretion': str, 'rdate': str, 'fdate': str, 'dsource': str}

pd_dtypes = {'cusip8': str, 'cusip9': str , 'titleOfClass': str, 'form': 'category', 'putCall': 'category',
           'shrsOrPrnAmt': 'category', 'value': 'float64', 'shares': 'float64', 'type': 'category', 'nameOfIssuer': str,
           'cik' : 'int64', 'address': 'category',  'dsource': 'category'}

# pd_dtypes_validation = {'cusip8': str, 'cusip9': str , 'titleOfClass': str, 'form': 'category', 'putCall': 'category',
#            'shrsOrPrnAmt': 'category', 'value': 'Int64', 'shares': 'Int64', 'type': 'category', 'nameOfIssuer': str,
#            'cik' : 'int64', 'address': 'category',  'dsource': 'category'}


# for file in processed_tables_copy.rglob("*.csv"):
#     new_file_name = file.parts[-2]+"-"+file.name
#     new_file_path = Path(os.path.join(TR_00_ALL_CSV_REDUCED, new_file_name))
#     if new_file_path.exists():  continue

    
#     schema = pl.scan_csv(file).schema
#     read_cols = list(set(schema.keys()).intersection(columns))
#     df = pl.read_csv(file, columns=read_cols, dtypes=dtypes)

#     for col in columns:
#         if col not in df.columns:
#             df = df.with_column(pl.lit(None, dtype=dtypes[col]).alias(col))

#     df = df.with_columns([pl.col("rdate").str.strptime(pl.Date, fmt="%Y%m%d"),
#                         pl.col("fdate").str.strptime(pl.Date, fmt="%Y%m%d"),
#                         pl.col("cusip8").str.to_uppercase(),
#                         pl.col("cusip9").str.to_uppercase(),
#                         pl.lit('dropbox').alias('dsource')]).select(columns)  

#     df.write_csv(new_file_path, sep=",")
#     df = df.to_pandas().astype(pd_dtypes_validation)

In [ ]:
# len(list(processed_tables_copy.rglob("*.csv"))), len(list(TR_00_ALL_CSV_REDUCED.glob("*.csv")))
# (308585, 308585)

### Working on selected **reduced csv** and **txt** files to manually improve the data quality with `pandera`

In [ ]:
selected_cik = [2230,3520,5272,7195,7789,9015,10742,14661,16972,18349,18748,19475,19617,21175,22657,24386,35442,35527,36066,36104,36644,36966,38777,39263,40417,40545,44365,45319,49205,50863,51762,51812,51964,52234,53417,59558,59951,60086,61227,67698,70858,71210,71259,72971,73124,80255,84616,89014,92230,93751,98758,102212,102909,105495,108572,200217,201772,216851,276101,310051,312348,313028,313807,314949,314957,314984,315014,315032,315038,315054,315066,315080,315157,315297,315498,316011,318989,320335,320376,351051,351173,351262,354204,356264,700529,704051,707179,712537,713676,714142,720672,723204,728083,728100,728618,732905,733020,740272,740913,741073,743127,750641,754811,757657,759944,762152,763212,763848,764068,764106,764112,764529,764532,765443,769317,769954,769963,775368,776867,778963,779519,788714,790354,790502,791191,791490,796848,799003,799004,801051,806097,807249,807985,808722,809339,809443,810265,810384,810386,810672,810716,811360,811454,813917,813933,814133,814375,816788,819535,820027,820123,820124,820289,820478,820743,821197,822581,823621,825293,829407,831001,831571,836372,837592,842782,842941,846222,846633,846788,846797,850401,850529,852743,854157,857508,859872,860486,860561,860580,860585,860643,860644,860645,860662,860748,860828,860857,861176,861177,861462,861787,862469,866361,866842,868491,869178,869179,869353,869367,872080,872163,872259,872573,872732,873630,874791,877134,877338,878228,881432,883511,883677,883782,883790,883803,883961,883965,884300,884314,884414,884423,884541,884546,884548,884566,884589,885062,885415,886982,887402,887777,887818,889232,891287,891478,893738,894205,894300,894309,895213,895421,897070,897378,897599,898358,898382,898399,898413,899211,900169,900529,900973,902219,902367,902464,902584,903064,903944,903947,903949,905567,905591,905608,906304,908195,909151,909661,911274,912938,914933,914976,915287,915325,916542,917579,918893,919079,919185,919192,919458,919489,919497,919530,919538,919859,920440,920441,921531,921669,922127,922439,922898,922940,923093,923116,923469,924166,924171,924181,926688,926833,926834,928047,928196,928566,928568,928633,930441,931097,932024,932974,933429,934639,934999,936698,936753,936936,936941,936944,937394,937522,937589,937615,937760,937886,938076,938206,938487,938582,938592,938759,939219,940445,941560,943719,944234,944804,945625,945631,947822,947996,948518,948669,949012,949509,949615,949623,949853,1000097,1000742,1002152,1002672,1002784,1004244,1005354,1005607,1005817,1006364,1006378,1006407,1006435,1007280,1007399,1007524,1008322,1008877,1008894,1008895,1008929,1008937,1009003,1009005,1009012,1009016,1009022,1009076,1009207,1009209,1009232,1009254,1009258,1009262,1010873,1010911,1011443,1011659,1013234,1013536,1013538,1013701,1014306,1014315,1014736,1014738,1015079,1015083,1015086,1015308,1016150,1016287,1016683,1016972,1017115,1017645,1017918,1018331,1018674,1018825,1019231,1020066,1020317,1020580,1020585,1020617,1020918,1021008,1021117,1021223,1021249,1021258,1021642,1021926,1023279,1024716,1025421,1026200,1026710,1027451,1027796,1027817,1029160,1030618,1030815,1031972,1032814,1033225,1033427,1033475,1033505,1033974,1033984,1034184,1034196,1034524,1034541,1034546,1034549,1034642,1034771,1034886,1035350,1035463,1035912,1036248,1036325,1037389,1037558,1037763,1037792,1038661,1039565,1039807,1040190,1040197,1040198,1040210,1040273,1040592,1040762,1041241,1041885,1042046,1044207,1044797,1044905,1044924,1044929,1044936,1046187,1047339,1048921,1049648,1049650,1050442,1050463,1050470,1051359,1052100,1053013,1053054,1053055,1054074,1054425,1054522,1054554,1054677,1055290,1055544,1055963,1055964,1055966,1056053,1056288,1056466,1056488,1056491,1056515,1056516,1056527,1056549,1056559,1056581,1056593,1056807,1056821,1056825,1056827,1056831,1056859,1056958,1056973,1057395,1057439,1058022,1058470,1058800,1059187,1061186,1061768,1062938,1065349,1065350,1066816,1067324,1067926,1067983,1068829,1070134,1071483,1072843,1074027,1074034,1074266,1074273,1076598,1077148,1077583,1078013,1078246,1078658,1078841,1079112,1079114,1079736,1079738,1079930,1080071,1080107,1080117,1080132,1080166,1080171,1080173,1080197,1080201,1080351,1080374,1080380,1080381,1080382,1080386,1080493,1080523,1080628,1080818,1081019,1081198,1082020,1082215,1082327,1082339,1082461,1082491,1082509,1082621,1082917,1083323,1083340,1084207,1084208,1084683,1085041,1085163,1085227,1085601,1085936,1086477,1086483,1086611,1086619,1086762,1086763,1088859,1088875,1088950,1089707,1089755,1089911,1089991,1090413,1091561,1091860,1091923,1092203,1092290,1092351,1092903,1093276,1093589,1094584,1094749,1095836,1096783,1097218,1097278,1097833,1100710,1101250,1102062,1102578,1102598,1103245,1103738,1103804,1103882,1103887,1104186,1104329,1104366,1105468,1105471,1105497,1105837,1105863,1105909,1106129,1106191,1106500,1106505,1106832,1107261,1107310,1108893,1108965,1108969,1109147,1110806,1113629,1114618,1114739,1114928,1115941,1116247,1125727,1125816,1129770,1133219,1134152,1140334,1140771,1142031,1142062,1158583,1389426,1398739,1469219]
len(selected_cik)

In [ ]:
# copy raw `TXT` filings for the selected 676 ciks (active in 1999 and 2000) to a separate folder `TR_00_TEST_676_CIK_TXT`
selected_cik = [2230,3520,5272,7195,7789,9015,10742,14661,16972,18349,18748,19475,19617,21175,22657,24386,35442,35527,36066,36104,36644,36966,38777,39263,40417,40545,44365,45319,49205,50863,51762,51812,51964,52234,53417,59558,59951,60086,61227,67698,70858,71210,71259,72971,73124,80255,84616,89014,92230,93751,98758,102212,102909,105495,108572,200217,201772,216851,276101,310051,312348,313028,313807,314949,314957,314984,315014,315032,315038,315054,315066,315080,315157,315297,315498,316011,318989,320335,320376,351051,351173,351262,354204,356264,700529,704051,707179,712537,713676,714142,720672,723204,728083,728100,728618,732905,733020,740272,740913,741073,743127,750641,754811,757657,759944,762152,763212,763848,764068,764106,764112,764529,764532,765443,769317,769954,769963,775368,776867,778963,779519,788714,790354,790502,791191,791490,796848,799003,799004,801051,806097,807249,807985,808722,809339,809443,810265,810384,810386,810672,810716,811360,811454,813917,813933,814133,814375,816788,819535,820027,820123,820124,820289,820478,820743,821197,822581,823621,825293,829407,831001,831571,836372,837592,842782,842941,846222,846633,846788,846797,850401,850529,852743,854157,857508,859872,860486,860561,860580,860585,860643,860644,860645,860662,860748,860828,860857,861176,861177,861462,861787,862469,866361,866842,868491,869178,869179,869353,869367,872080,872163,872259,872573,872732,873630,874791,877134,877338,878228,881432,883511,883677,883782,883790,883803,883961,883965,884300,884314,884414,884423,884541,884546,884548,884566,884589,885062,885415,886982,887402,887777,887818,889232,891287,891478,893738,894205,894300,894309,895213,895421,897070,897378,897599,898358,898382,898399,898413,899211,900169,900529,900973,902219,902367,902464,902584,903064,903944,903947,903949,905567,905591,905608,906304,908195,909151,909661,911274,912938,914933,914976,915287,915325,916542,917579,918893,919079,919185,919192,919458,919489,919497,919530,919538,919859,920440,920441,921531,921669,922127,922439,922898,922940,923093,923116,923469,924166,924171,924181,926688,926833,926834,928047,928196,928566,928568,928633,930441,931097,932024,932974,933429,934639,934999,936698,936753,936936,936941,936944,937394,937522,937589,937615,937760,937886,938076,938206,938487,938582,938592,938759,939219,940445,941560,943719,944234,944804,945625,945631,947822,947996,948518,948669,949012,949509,949615,949623,949853,1000097,1000742,1002152,1002672,1002784,1004244,1005354,1005607,1005817,1006364,1006378,1006407,1006435,1007280,1007399,1007524,1008322,1008877,1008894,1008895,1008929,1008937,1009003,1009005,1009012,1009016,1009022,1009076,1009207,1009209,1009232,1009254,1009258,1009262,1010873,1010911,1011443,1011659,1013234,1013536,1013538,1013701,1014306,1014315,1014736,1014738,1015079,1015083,1015086,1015308,1016150,1016287,1016683,1016972,1017115,1017645,1017918,1018331,1018674,1018825,1019231,1020066,1020317,1020580,1020585,1020617,1020918,1021008,1021117,1021223,1021249,1021258,1021642,1021926,1023279,1024716,1025421,1026200,1026710,1027451,1027796,1027817,1029160,1030618,1030815,1031972,1032814,1033225,1033427,1033475,1033505,1033974,1033984,1034184,1034196,1034524,1034541,1034546,1034549,1034642,1034771,1034886,1035350,1035463,1035912,1036248,1036325,1037389,1037558,1037763,1037792,1038661,1039565,1039807,1040190,1040197,1040198,1040210,1040273,1040592,1040762,1041241,1041885,1042046,1044207,1044797,1044905,1044924,1044929,1044936,1046187,1047339,1048921,1049648,1049650,1050442,1050463,1050470,1051359,1052100,1053013,1053054,1053055,1054074,1054425,1054522,1054554,1054677,1055290,1055544,1055963,1055964,1055966,1056053,1056288,1056466,1056488,1056491,1056515,1056516,1056527,1056549,1056559,1056581,1056593,1056807,1056821,1056825,1056827,1056831,1056859,1056958,1056973,1057395,1057439,1058022,1058470,1058800,1059187,1061186,1061768,1062938,1065349,1065350,1066816,1067324,1067926,1067983,1068829,1070134,1071483,1072843,1074027,1074034,1074266,1074273,1076598,1077148,1077583,1078013,1078246,1078658,1078841,1079112,1079114,1079736,1079738,1079930,1080071,1080107,1080117,1080132,1080166,1080171,1080173,1080197,1080201,1080351,1080374,1080380,1080381,1080382,1080386,1080493,1080523,1080628,1080818,1081019,1081198,1082020,1082215,1082327,1082339,1082461,1082491,1082509,1082621,1082917,1083323,1083340,1084207,1084208,1084683,1085041,1085163,1085227,1085601,1085936,1086477,1086483,1086611,1086619,1086762,1086763,1088859,1088875,1088950,1089707,1089755,1089911,1089991,1090413,1091561,1091860,1091923,1092203,1092290,1092351,1092903,1093276,1093589,1094584,1094749,1095836,1096783,1097218,1097278,1097833,1100710,1101250,1102062,1102578,1102598,1103245,1103738,1103804,1103882,1103887,1104186,1104329,1104366,1105468,1105471,1105497,1105837,1105863,1105909,1106129,1106191,1106500,1106505,1106832,1107261,1107310,1108893,1108965,1108969,1109147,1110806,1113629,1114618,1114739,1114928,1115941,1116247,1125727,1125816,1129770,1133219,1134152,1140334,1140771,1142031,1142062,1158583,1389426,1398739,1469219]

from pathlib import Path
import shutil

# Define the paths
# main_directory = Path('E:\\app_data\\sec_apps_data\\speed_test\\filings_13f_full')
# filings_directory = main_directory / 'filings'
# selected_filings_directory = Path(r"E:\app_data\dropbox_13f_files\processed_tables\TR_00_TEST_676_CIK_TXT")

# # Create the selected_filings_directory if it doesn't exist
# selected_filings_directory.mkdir(parents=True, exist_ok=True)

# # Iterate through the subdirectories in the filings directory
# for subdir in filings_directory.iterdir():
#     if int(subdir.name) in selected_cik:
#         # Copy the subdirectory to the selected_filings_directory
#         shutil.copytree(subdir, selected_filings_directory / subdir.name)

In [ ]:
# copy reduced `CSV` filings for the selected 676 ciks (active in 1999 and 2000) to a separate folder `TR_01_TEST_676_CIK_CSV_REDUCED`
selected_cik = [2230,3520,5272,7195,7789,9015,10742,14661,16972,18349,18748,19475,19617,21175,22657,24386,35442,35527,36066,36104,36644,36966,38777,39263,40417,40545,44365,45319,49205,50863,51762,51812,51964,52234,53417,59558,59951,60086,61227,67698,70858,71210,71259,72971,73124,80255,84616,89014,92230,93751,98758,102212,102909,105495,108572,200217,201772,216851,276101,310051,312348,313028,313807,314949,314957,314984,315014,315032,315038,315054,315066,315080,315157,315297,315498,316011,318989,320335,320376,351051,351173,351262,354204,356264,700529,704051,707179,712537,713676,714142,720672,723204,728083,728100,728618,732905,733020,740272,740913,741073,743127,750641,754811,757657,759944,762152,763212,763848,764068,764106,764112,764529,764532,765443,769317,769954,769963,775368,776867,778963,779519,788714,790354,790502,791191,791490,796848,799003,799004,801051,806097,807249,807985,808722,809339,809443,810265,810384,810386,810672,810716,811360,811454,813917,813933,814133,814375,816788,819535,820027,820123,820124,820289,820478,820743,821197,822581,823621,825293,829407,831001,831571,836372,837592,842782,842941,846222,846633,846788,846797,850401,850529,852743,854157,857508,859872,860486,860561,860580,860585,860643,860644,860645,860662,860748,860828,860857,861176,861177,861462,861787,862469,866361,866842,868491,869178,869179,869353,869367,872080,872163,872259,872573,872732,873630,874791,877134,877338,878228,881432,883511,883677,883782,883790,883803,883961,883965,884300,884314,884414,884423,884541,884546,884548,884566,884589,885062,885415,886982,887402,887777,887818,889232,891287,891478,893738,894205,894300,894309,895213,895421,897070,897378,897599,898358,898382,898399,898413,899211,900169,900529,900973,902219,902367,902464,902584,903064,903944,903947,903949,905567,905591,905608,906304,908195,909151,909661,911274,912938,914933,914976,915287,915325,916542,917579,918893,919079,919185,919192,919458,919489,919497,919530,919538,919859,920440,920441,921531,921669,922127,922439,922898,922940,923093,923116,923469,924166,924171,924181,926688,926833,926834,928047,928196,928566,928568,928633,930441,931097,932024,932974,933429,934639,934999,936698,936753,936936,936941,936944,937394,937522,937589,937615,937760,937886,938076,938206,938487,938582,938592,938759,939219,940445,941560,943719,944234,944804,945625,945631,947822,947996,948518,948669,949012,949509,949615,949623,949853,1000097,1000742,1002152,1002672,1002784,1004244,1005354,1005607,1005817,1006364,1006378,1006407,1006435,1007280,1007399,1007524,1008322,1008877,1008894,1008895,1008929,1008937,1009003,1009005,1009012,1009016,1009022,1009076,1009207,1009209,1009232,1009254,1009258,1009262,1010873,1010911,1011443,1011659,1013234,1013536,1013538,1013701,1014306,1014315,1014736,1014738,1015079,1015083,1015086,1015308,1016150,1016287,1016683,1016972,1017115,1017645,1017918,1018331,1018674,1018825,1019231,1020066,1020317,1020580,1020585,1020617,1020918,1021008,1021117,1021223,1021249,1021258,1021642,1021926,1023279,1024716,1025421,1026200,1026710,1027451,1027796,1027817,1029160,1030618,1030815,1031972,1032814,1033225,1033427,1033475,1033505,1033974,1033984,1034184,1034196,1034524,1034541,1034546,1034549,1034642,1034771,1034886,1035350,1035463,1035912,1036248,1036325,1037389,1037558,1037763,1037792,1038661,1039565,1039807,1040190,1040197,1040198,1040210,1040273,1040592,1040762,1041241,1041885,1042046,1044207,1044797,1044905,1044924,1044929,1044936,1046187,1047339,1048921,1049648,1049650,1050442,1050463,1050470,1051359,1052100,1053013,1053054,1053055,1054074,1054425,1054522,1054554,1054677,1055290,1055544,1055963,1055964,1055966,1056053,1056288,1056466,1056488,1056491,1056515,1056516,1056527,1056549,1056559,1056581,1056593,1056807,1056821,1056825,1056827,1056831,1056859,1056958,1056973,1057395,1057439,1058022,1058470,1058800,1059187,1061186,1061768,1062938,1065349,1065350,1066816,1067324,1067926,1067983,1068829,1070134,1071483,1072843,1074027,1074034,1074266,1074273,1076598,1077148,1077583,1078013,1078246,1078658,1078841,1079112,1079114,1079736,1079738,1079930,1080071,1080107,1080117,1080132,1080166,1080171,1080173,1080197,1080201,1080351,1080374,1080380,1080381,1080382,1080386,1080493,1080523,1080628,1080818,1081019,1081198,1082020,1082215,1082327,1082339,1082461,1082491,1082509,1082621,1082917,1083323,1083340,1084207,1084208,1084683,1085041,1085163,1085227,1085601,1085936,1086477,1086483,1086611,1086619,1086762,1086763,1088859,1088875,1088950,1089707,1089755,1089911,1089991,1090413,1091561,1091860,1091923,1092203,1092290,1092351,1092903,1093276,1093589,1094584,1094749,1095836,1096783,1097218,1097278,1097833,1100710,1101250,1102062,1102578,1102598,1103245,1103738,1103804,1103882,1103887,1104186,1104329,1104366,1105468,1105471,1105497,1105837,1105863,1105909,1106129,1106191,1106500,1106505,1106832,1107261,1107310,1108893,1108965,1108969,1109147,1110806,1113629,1114618,1114739,1114928,1115941,1116247,1125727,1125816,1129770,1133219,1134152,1140334,1140771,1142031,1142062,1158583,1389426,1398739,1469219]

from pathlib import Path
import shutil

# Define the paths
TR_00_ALL_CSV_REDUCED = Path(r'E:\app_data\dropbox_13f_files\processed_tables\TR_00_ALL_CSV_REDUCED')
TR_01_TEST_676_CIK_CSV_REDUCED = Path(r"E:\app_data\dropbox_13f_files\processed_tables\TR_01_TEST_676_CIK_CSV_REDUCED")

# Create the selected_filings_directory if it doesn't exist
TR_01_TEST_676_CIK_CSV_REDUCED.mkdir(parents=True, exist_ok=True)

# Iterate through the subdirectories in the filings directory
# for file in TR_00_ALL_CSV_REDUCED.glob("*.csv"):
#     # file.name
#     # if subdir.name.isdigit() and int(subdir.name) in selected_cik:
#     if int(subdir.name) in selected_cik:
#         # Copy the subdirectory to the selected_filings_directory
#         shutil.copytree(subdir, selected_filings_directory / subdir.name)

In [ ]:
len(clean_processed_files) # 1st round 8204

### Running `pandera` validation through the **selected** cik

In [ ]:
cleaned_dfs = []
clean_processed_files = []
files_failures = []
bad_dfs =[]

In [ ]:
origin_clean = [file.name for file in TR_01_TEST_676_CIK_CSV_ORIGIN_CLEAN.glob("*.csv")]
files_failures = [file.name.split("-", maxsplit=1)[-1] for file in TR_01_FAILURE_CASES_CSV.glob("*.csv")]
cleaned = [file.name for file in TR_01_TEST_676_CIK_CSV_CLEANED.glob("*.csv")]

In [ ]:
origin_clean, files_failures, cleaned

In [ ]:
%%time

from pathlib import Path
import shutil
import pandas as pd
import polars as pl
import pandera as pa

selected_cik = [2230,3520,5272,7195,7789,9015,10742,14661,16972,18349,18748,19475,19617,21175,22657,24386,35442,35527,36066,36104,36644,36966,38777,39263,40417,40545,44365,45319,49205,50863,51762,51812,51964,52234,53417,59558,59951,60086,61227,67698,70858,71210,71259,72971,73124,80255,84616,89014,92230,93751,98758,102212,102909,105495,108572,200217,201772,216851,276101,310051,312348,313028,313807,314949,314957,314984,315014,315032,315038,315054,315066,315080,315157,315297,315498,316011,318989,320335,320376,351051,351173,351262,354204,356264,700529,704051,707179,712537,713676,714142,720672,723204,728083,728100,728618,732905,733020,740272,740913,741073,743127,750641,754811,757657,759944,762152,763212,763848,764068,764106,764112,764529,764532,765443,769317,769954,769963,775368,776867,778963,779519,788714,790354,790502,791191,791490,796848,799003,799004,801051,806097,807249,807985,808722,809339,809443,810265,810384,810386,810672,810716,811360,811454,813917,813933,814133,814375,816788,819535,820027,820123,820124,820289,820478,820743,821197,822581,823621,825293,829407,831001,831571,836372,837592,842782,842941,846222,846633,846788,846797,850401,850529,852743,854157,857508,859872,860486,860561,860580,860585,860643,860644,860645,860662,860748,860828,860857,861176,861177,861462,861787,862469,866361,866842,868491,869178,869179,869353,869367,872080,872163,872259,872573,872732,873630,874791,877134,877338,878228,881432,883511,883677,883782,883790,883803,883961,883965,884300,884314,884414,884423,884541,884546,884548,884566,884589,885062,885415,886982,887402,887777,887818,889232,891287,891478,893738,894205,894300,894309,895213,895421,897070,897378,897599,898358,898382,898399,898413,899211,900169,900529,900973,902219,902367,902464,902584,903064,903944,903947,903949,905567,905591,905608,906304,908195,909151,909661,911274,912938,914933,914976,915287,915325,916542,917579,918893,919079,919185,919192,919458,919489,919497,919530,919538,919859,920440,920441,921531,921669,922127,922439,922898,922940,923093,923116,923469,924166,924171,924181,926688,926833,926834,928047,928196,928566,928568,928633,930441,931097,932024,932974,933429,934639,934999,936698,936753,936936,936941,936944,937394,937522,937589,937615,937760,937886,938076,938206,938487,938582,938592,938759,939219,940445,941560,943719,944234,944804,945625,945631,947822,947996,948518,948669,949012,949509,949615,949623,949853,1000097,1000742,1002152,1002672,1002784,1004244,1005354,1005607,1005817,1006364,1006378,1006407,1006435,1007280,1007399,1007524,1008322,1008877,1008894,1008895,1008929,1008937,1009003,1009005,1009012,1009016,1009022,1009076,1009207,1009209,1009232,1009254,1009258,1009262,1010873,1010911,1011443,1011659,1013234,1013536,1013538,1013701,1014306,1014315,1014736,1014738,1015079,1015083,1015086,1015308,1016150,1016287,1016683,1016972,1017115,1017645,1017918,1018331,1018674,1018825,1019231,1020066,1020317,1020580,1020585,1020617,1020918,1021008,1021117,1021223,1021249,1021258,1021642,1021926,1023279,1024716,1025421,1026200,1026710,1027451,1027796,1027817,1029160,1030618,1030815,1031972,1032814,1033225,1033427,1033475,1033505,1033974,1033984,1034184,1034196,1034524,1034541,1034546,1034549,1034642,1034771,1034886,1035350,1035463,1035912,1036248,1036325,1037389,1037558,1037763,1037792,1038661,1039565,1039807,1040190,1040197,1040198,1040210,1040273,1040592,1040762,1041241,1041885,1042046,1044207,1044797,1044905,1044924,1044929,1044936,1046187,1047339,1048921,1049648,1049650,1050442,1050463,1050470,1051359,1052100,1053013,1053054,1053055,1054074,1054425,1054522,1054554,1054677,1055290,1055544,1055963,1055964,1055966,1056053,1056288,1056466,1056488,1056491,1056515,1056516,1056527,1056549,1056559,1056581,1056593,1056807,1056821,1056825,1056827,1056831,1056859,1056958,1056973,1057395,1057439,1058022,1058470,1058800,1059187,1061186,1061768,1062938,1065349,1065350,1066816,1067324,1067926,1067983,1068829,1070134,1071483,1072843,1074027,1074034,1074266,1074273,1076598,1077148,1077583,1078013,1078246,1078658,1078841,1079112,1079114,1079736,1079738,1079930,1080071,1080107,1080117,1080132,1080166,1080171,1080173,1080197,1080201,1080351,1080374,1080380,1080381,1080382,1080386,1080493,1080523,1080628,1080818,1081019,1081198,1082020,1082215,1082327,1082339,1082461,1082491,1082509,1082621,1082917,1083323,1083340,1084207,1084208,1084683,1085041,1085163,1085227,1085601,1085936,1086477,1086483,1086611,1086619,1086762,1086763,1088859,1088875,1088950,1089707,1089755,1089911,1089991,1090413,1091561,1091860,1091923,1092203,1092290,1092351,1092903,1093276,1093589,1094584,1094749,1095836,1096783,1097218,1097278,1097833,1100710,1101250,1102062,1102578,1102598,1103245,1103738,1103804,1103882,1103887,1104186,1104329,1104366,1105468,1105471,1105497,1105837,1105863,1105909,1106129,1106191,1106500,1106505,1106832,1107261,1107310,1108893,1108965,1108969,1109147,1110806,1113629,1114618,1114739,1114928,1115941,1116247,1125727,1125816,1129770,1133219,1134152,1140334,1140771,1142031,1142062,1158583,1389426,1398739,1469219]
TR_01_TEST_676_CIK_CSV_REDUCED = Path(r"E:\app_data\dropbox_13f_files\processed_tables\TR_01_TEST_676_CIK_CSV_REDUCED")
TR_01_TEST_676_CIK_CSV_ORIGIN_CLEAN = Path(r"E:\app_data\dropbox_13f_files\processed_tables\TR_01_TEST_676_CIK_CSV_ORIGIN_CLEAN")
TR_01_TEST_676_CIK_CSV_CLEANED = Path(r"E:\app_data\dropbox_13f_files\processed_tables\TR_01_TEST_676_CIK_CSV_CLEANED")
TR_01_FAILURE_CASES_CSV = Path(r"E:\app_data\dropbox_13f_files\processed_tables\TR_01_FAILURE_CASES_CSV")

filings_676_txt = Path(r"E:\app_data\dropbox_13f_files\processed_tables\TR_00_TEST_676_CIK_TXT")

columns = ['cik', 'cusip9','value', 'shares','rdate', 'fdate',\
           'address', 'form', 'shrsOrPrnAmt', 'putCall', 'nameOfIssuer', 'titleOfClass', 'type', 'dsource']

pl_dtypes = {'cusip8': str, 'cusip9': str , 'titleOfClass': str, 'form': str, 'putCall': str,
           'shrsOrPrnAmt': str, 'value': pl.Float64, 'shares': pl.Float64, 'type': str, 'nameOfIssuer': str,
           'cik' : pl.Int64, 'address': str,  'dsource': str}

pd_dtypes_validation = {'cusip9': str , 'titleOfClass': str, 'form': 'category',
           'putCall': 'category', 'shrsOrPrnAmt': 'category', 'value': 'float64',
           'shares': 'float64', 'type': 'category', 'nameOfIssuer': str,
           'cik' : 'int64', 'address': 'category',  'dsource': 'category'}

cusip_eq_9_schema = pa.DataFrameSchema({
    "cusip9": pa.Column(str, pa.Check(lambda s: s.str.len() == 9), required=True, nullable=False) ,
    "value":  pa.Column(float, pa.Check(lambda s: s != 0.0), required=True, nullable=False),
    "shares": pa.Column(float, pa.Check(lambda s: s != 0.0), required=True, nullable=False)
})

cleaned = [file.name for file in TR_01_TEST_676_CIK_CSV_CLEANED.glob("*.csv")]
# bad_dfs =[]
origin_clean = [file.name for file in TR_01_TEST_676_CIK_CSV_ORIGIN_CLEAN.glob("*.csv")]
files_failures = [file.name.split("-", maxsplit=1)[-1] for file in TR_01_FAILURE_CASES_CSV.glob("*.csv")]

for index, cik in enumerate(selected_cik):
    for file in TR_01_TEST_676_CIK_CSV_REDUCED.glob(f"{cik}-*.csv"):
        if file.name not in set(origin_clean + files_failures + cleaned):
            try:
                df = pl.read_csv(file, columns=columns, dtypes=pl_dtypes, parse_dates=True) 
            except Exception as e:
                print(f"Problem reading file... {file.name}")
                print(e)
                # os.system(f"code "+ file.as_posix())
                # i = input("did you correct the file?")
                # if i == "yes":
                #     print(f"file: {file} is finished")
                #     continue

                # else: 
                #     sys.exit()
            df = df.to_pandas().astype(pd_dtypes_validation)

            try:
                cusip_eq_9_schema.validate(df[columns], lazy=True)
                df.to_csv(Path.joinpath(TR_01_TEST_676_CIK_CSV_ORIGIN_CLEAN, file.name), index=False)
                # print(f"Originally Clean: {file.name}")
            except pa.errors.SchemaErrors as e:
                failure_cases = e.failure_cases
                failure_cases = (failure_cases.assign(df_file=file,
                                                     df_cik=cik,
                                                     df_rdate=df.rdate,
                                                     df_fdate=df.fdate,
                                                     df_value=df.value,
                                                     df_shares=df.shares))
                failure_cases.to_csv(Path.joinpath(TR_01_FAILURE_CASES_CSV, f"bad-{file.name}"), index=False)
                # bad_dfs.append(failure_cases)
                cleaned_df = df[~df.index.isin(failure_cases["index"])]
                # !!! TODO !!! Now empty cleaned_df get written to `csv` files. Need to add/test code
                # the code below!!!
                # if not cleaned_df.empty:
                #     cleaned_df.to_csv(Path.joinpath(TR_01_TEST_676_CIK_CSV_CLEANED, file.name), index=False)
                # print(f"CIK: {cik}. Year: {file.name.split('-')[2]}, File: {file.name}.")
                # print(f"Total rows: {df.shape[0]}. Bad rows: {failure_cases.shape[0]}. Clean rows: {cleaned_df.shape[0]}")
                # print(f"{round((failure_cases.shape[0] / df.shape[0]) * 100, 2)}% bad rows", end='\n#############\n')
                # cleaned_dfs.append(cleaned_df)
    df, failure_cases, cleaned_df = (None, None, None)

        

        # dfs.append(df)
        # df = pl.concat(dfs).to_pandas().astype(pd_dtypes_validation)
        # df = df.to_pandas().astype(pd_dtypes_validation)

### Running `pandera` validation through the **ALL** reduced `.csv` with `fdate` <= '2014-01-01'
* `csv` filings from **dropbox** will be used only for the range from the `absolute beginnig` to `2013-12-31`
* `parquet` filings downloaded and parsed by my called **sec_app** will be used from `2014-01-01` and on

In [ ]:
%%time
from pathlib import Path
import pandas as pd
import polars as pl
import pandera as pa

TR_01_ALL_CSV_CLEANED = Path(r'/Users/yo_macbook/Documents/app_data/dropbox_13f_files/processed_tables/TR_01_ALL_CSV_CLEANED')

TR_01_ALL_CSV_REDUCED = \
    Path(r'/Users/yo_macbook/Documents/app_data/dropbox_13f_files/processed_tables/TR_01_ALL_CSV_REDUCED')
TR_01_ALL_CSV_CLEANED = \
    Path(r"/Users/yo_macbook/Documents/app_data/dropbox_13f_files/processed_tables/TR_01_ALL_CSV_CLEANED")
TR_01_FAILURE_CASES_CSV = \
    Path(r"/Users/yo_macbook/Documents/app_data/dropbox_13f_files/processed_tables/TR_01_FAILURE_CASES_CSV")

columns = ['cik', 'cusip9','value', 'shares','rdate', 'fdate',\
           'address', 'form', 'shrsOrPrnAmt', 'putCall', 'nameOfIssuer', 'titleOfClass', 'type', 'dsource']

pl_dtypes = {'cusip9': str , 'titleOfClass': str, 'form': str, 'putCall': str,
           'shrsOrPrnAmt': str, 'value': pl.Float64, 'shares': pl.Float64, 'type': str, 'nameOfIssuer': str,
           'cik' : pl.Int64, 'address': str,  'dsource': str}

pd_dtypes_validation = {'cusip9': str , 'titleOfClass': str, 'form': 'category',
           'putCall': 'category', 'shrsOrPrnAmt': 'category', 'value': 'float64',
           'shares': 'float64', 'type': 'category', 'nameOfIssuer': str,
           'cik' : 'int64', 'address': 'category',  'dsource': 'category'}
# pandera schema
validation_schema = pa.DataFrameSchema({
    "cusip9": pa.Column(str,
                       pa.Check(lambda s: s.str.len() == 9),
                       required=True, nullable=False),
    "value":  pa.Column(float, pa.Check(lambda s: s != 0.0),required=True, nullable=False),
    "shares": pa.Column(float, pa.Check(lambda s: s != 0.0), required=True, nullable=False)
        })

cleaned = [file.name for file in TR_01_ALL_CSV_CLEANED.glob("*.csv")]
files_failures = [file.name.split("-", maxsplit=1)[-1] for file in TR_01_FAILURE_CASES_CSV.glob("*.csv")]
cutoff_date = '2014-01-01'

for file in TR_01_ALL_CSV_REDUCED.glob(f"*.csv"):
    if file.name not in set(files_failures + cleaned):
        try:
            df = pl.read_csv(file, columns=columns, dtypes=pl_dtypes, parse_dates=True) 
        except Exception as e:
            print(f"Problem reading file... {file.name}")
            print(e)
        df = df.to_pandas().astype(pd_dtypes_validation)
        if df.fdate.max()  >= pd.to_datetime(cutoff_date,format='%Y-%m-%d'): continue

        try:
            validation_schema.validate(df, lazy=True)
            df.to_csv(Path.joinpath(TR_01_ALL_CSV_CLEANED, file.name), index=False)
            # print(f"Originally Clean: {file.name}")
        except pa.errors.SchemaErrors as e:
            failure_cases = e.failure_cases
            failure_cases = (failure_cases.assign(df_file=file,
                                                 df_cik=df.cik,
                                                 df_rdate=df.rdate,
                                                 df_fdate=df.fdate)).astype({'failure_case':str})
            failure_cases.to_csv(Path.joinpath(TR_01_FAILURE_CASES_CSV, f"bad-{file.name}"), index=False)
            # bad_dfs.append(failure_cases)
            cleaned_df = df[~df.index.isin(failure_cases["index"])]
            # !!! TODO !!! Now empty cleaned_df get written to `csv` files. Need to add/test code
            # the code below!!!
            if not cleaned_df.empty:
                cleaned_df.to_csv(Path.joinpath(TR_01_ALL_CSV_CLEANED, file.name), index=False)

df, failure_cases, cleaned_df = (None, None, None)


### Move cleaned `csv` for 676 cik from `TR_01_TEST_676_CIK_CSV_CLEANED_BOTH` to `TR_01_ALL_CSV_CLEANED` with all `.csv`
* **Only** the filings before **`2014-01-01`** are moved
* It's one-off code to move already cleaned filings for 676 cik that were active between 1999 and 2020 to the common folder with all `csv` that were passed through the `pandera` data quality tool

In [ ]:
%%time
from pathlib import Path
import pandas as pd
import polars as pl
import pandera as pa

TR_01_ALL_CSV_CLEANED = \
    Path(r"/Users/yo_macbook/Documents/app_data/dropbox_13f_files/processed_tables/TR_01_ALL_CSV_CLEANED")

TR_01_TEST_676_CIK_CSV_CLEANED_BOTH = \
    Path(r'/Users/yo_macbook/Documents/app_data/dropbox_13f_files/processed_tables/TR_01_TEST_676_CIK_CSV_CLEANED_BOTH')

columns = ['cik', 'cusip9','value', 'shares','rdate', 'fdate',\
           'address', 'form', 'shrsOrPrnAmt', 'putCall', 'nameOfIssuer', 'titleOfClass', 'type', 'dsource']

pl_dtypes = {'cusip9': str , 'titleOfClass': str, 'form': str, 'putCall': str,
           'shrsOrPrnAmt': str, 'value': pl.Float64, 'shares': pl.Float64, 'type': str, 'nameOfIssuer': str,
           'cik' : pl.Int64, 'address': str,  'dsource': str}

pd_dtypes_validation = {'cusip9': str , 'titleOfClass': str, 'form': 'category',
           'putCall': 'category', 'shrsOrPrnAmt': 'category', 'value': 'float64',
           'shares': 'float64', 'type': 'category', 'nameOfIssuer': str,
           'cik' : 'int64', 'address': 'category',  'dsource': 'category'}

cutoff_date = '2014-01-01'

for file in TR_01_TEST_676_CIK_CSV_CLEANED_BOTH.glob(f"*.csv"):
    # print(file.name)
    df = pl.read_csv(file, columns=columns, dtypes=pl_dtypes, parse_dates=True) 
    df = df.to_pandas().astype(pd_dtypes_validation)
    # print(df.head(2))
    if df.empty: continue
    if df.fdate.max()  >= pd.to_datetime(cutoff_date,format='%Y-%m-%d'): continue 
    df.to_csv(Path.joinpath(TR_01_ALL_CSV_CLEANED, file.name), index=False)


### Move cleaned `parquet` for 676 cik from `TR_02_TEST_676_CIK_PARQ_SEC_APP` to `TR_02_ALL_PARQ_SEC_APP` with all `parquet`
* **Only** the filings from **`2014-01-01`** and onwards are moved
* It's a one-off code to move already cleaned filings for 676 cik that were active between 1999 and 2020 to the common folder with all `parquet` that were passed through the `pandera` data quality tool

In [ ]:
%%time
from pathlib import Path
import pandas as pd
import polars as pl
import shutil

TR_02_ALL_PARQ_SEC_APP = \
    Path(r'/Users/yo_macbook/Documents/app_data/sec_apps_data/TR_02_ALL_PARQ_SEC_APP')

TR_02_TEST_676_CIK_PARQ_SEC_APP = \
    Path(r'/Users/yo_macbook/Documents/app_data/sec_apps_data/TR_02_TEST_676_CIK_PARQ_SEC_APP')

columns_sec_app = ['cik', 'cusip','value', 'shares','rdate', 'fdate',\
            'accession_number', 'submission_type', 'type', 'dsource', 'file']

pd_dtypes_validation_sec_app = {'cusip': str, 'submission_type': 'category', 'value': 'float64', 
            'shares': 'float64', 'type': 'category', 'cik' : 'int64',
            'accession_number': 'category',  'dsource': 'category', 'file':'category'}

pl_dtypes_sec_app = {'cusip': str ,'submission_type': str, 'value': pl.Float64, 'shares': pl.Float64, 'type': str,
            'cik' : pl.Int64, 'accession_number': str,  'dsource': str, 'file': str}

cutoff_date = '2014-01-01'

for file in TR_02_TEST_676_CIK_PARQ_SEC_APP.glob(f"*.parquet"):
    # print(file.name)
    df = pl.read_parquet(file, columns=columns_sec_app)
    df = df.to_pandas().astype(pd_dtypes_validation_sec_app)
    # print(df.head(2))
    if df.empty: continue
    if df.fdate.max() < pd.to_datetime(cutoff_date,format='%Y-%m-%d'): continue 
    else: shutil.copy(file, Path.joinpath(TR_02_ALL_PARQ_SEC_APP, file.name))

### Remove entries with `cusip` containing 'TEGIES' from every file, `csv` and `parquet`

In [ ]:
set(list(dropbox_csv.glob('*.csv')) + list(sec_app_parq.glob('*.parquet')))

In [ ]:

import pandas as pd
import polars as pl
from pathlib import Path

columns = ['cik', 'cusip9','value', 'shares','rdate', 'fdate',\
            'address', 'form', 'shrsOrPrnAmt', 'putCall', 'nameOfIssuer', 'titleOfClass', 'type', 'dsource']

pl_dtypes = {'cusip8': str, 'cusip9': str , 'titleOfClass': str, 'form': str, 'putCall': str,
            'shrsOrPrnAmt': str, 'value': pl.Float64, 'shares': pl.Float64, 'type': str, 'nameOfIssuer': str,
            'cik' : pl.Int64, 'address': str,  'dsource': str}

pd_dtypes_validation = {'cusip9': str , 'titleOfClass': str, 'form': 'category',
            'putCall': 'category', 'shrsOrPrnAmt': 'category', 'value': 'float64',
            'shares': 'float64', 'type': 'category', 'nameOfIssuer': str,
            'cik' : 'int64', 'address': 'category',  'dsource': 'category'}

def remove_cusip_w_substring(dir_csv, dir_parq):
    """
    input: file directories, csv and parquet
    returns: removes cusip containing certain substring and 
    saves it back to the original files
    csv or parquet
    
    """
    for file in set(list(dropbox_csv.glob('*.csv')) + list(sec_app_parq.glob('*.parquet'))):
        try:
            if file.suffix == '.parquet' and file.exists():
                df = pd.read_parquet(file)
                if not df.query('cusip.str.contains("TEGIES")').empty: 
                    print(f'found TEGIES in {file} ')
                    df = df.drop(df.query('cusip.str.contains("TEGIES")').index)
                    df.to_parquet(file, index=False)
                    print('saved new df to parquet', file)
            elif file.suffix == '.csv' and file.exists():
                df = pl.read_csv(file, columns=columns, dtypes=pl_dtypes, parse_dates=True) 
                df = df.to_pandas().astype(pd_dtypes_validation)
                if not df.query('cusip9.str.contains("TEGIES")').empty: 
                    print(f'found TEGIES in {file} ')
                    df = df.drop(df.query('cusip9.str.contains("TEGIES")').index)
                    df.to_csv(file, index=False)
                    print('saved new df to csv', file)
        except Exception as e:
            print(f"An error occurred: {e}")


In [ ]:
%%time
dropbox_csv = Path(r"/Users/yo_macbook/Documents/app_data/dropbox_13f_files/processed_tables/TR_01_ALL_CSV_CLEANED")
sec_app_parq = Path(r'/Users/yo_macbook/Documents/app_data/sec_apps_data/TR_02_ALL_PARQ_SEC_APP')

# remove_cusip_w_substring(dropbox_csv, sec_app_parq)

## After the code above generated new set of `csv` we manually review and clean the obvious ones
* It's done in the notebook `cik_cleaning_tool.ipynb`
* It was a very long manual process where both `csv` and `parquet` filings were connected in the data cleaning viz and manually cleaned as much as possible. The typical errors are:
     - non division by 1000. Now this big dataset is mostly made of shares divided by 1000
     - swapped columns for `shares` and `value`. 
     - bad `value` in general

### Here we take the cleaned `csv` files and convert them to `parquet`
* We're adding new column `data_load` with default value `manual_csv_2023_01_01_01_01`. It means this is the data that was manually dealt with and cleaned. It's the first baseline and a one-off task. 
* The data is from the very beginning to the 'fdate' < `2014-01-01`
* All the rest will come from the manually cleaned `parquet` filings
* !!! **IMPORTANT** !!! this code multiplies manually cleaned `csv` value by 1000

In [ ]:
%%time
from pathlib import Path
import pandas as pd
import polars as pl

def clean_csv_to_parquet(csv_folder, parq_folder):
    """
    !!! IMPORTANT !!! this code multiplies manually cleaned csv value by 1000
    input: list of csv file paths to change a bit and convert to parquet
    returns: enriched parquet from cleaned csv
    """
    columns = ['cik', 'cusip9','value', 'shares','rdate', 'fdate',\
            'address', 'form', 'shrsOrPrnAmt', 'putCall', 'nameOfIssuer', 'titleOfClass', 'type', 'dsource']

    pl_dtypes = {'cik' : pl.Int64, 'cusip9': str, 'value': pl.Float64, 'shares': pl.Float64, 'address': str,
                 'form': str, 'shrsOrPrnAmt': str, 'putCall': str, 'nameOfIssuer': str, 'titleOfClass': str,
                 'type': str, 'dsource': str}

    pd_dtypes_validation = {'cusip9': str , 'titleOfClass': str, 'form': 'category',
                'putCall': 'category', 'shrsOrPrnAmt': 'category', 'value': 'float64',
                'shares': 'float64', 'type': 'category', 'nameOfIssuer': str,
                'cik' : 'int64', 'address': 'category',  'dsource': 'category'}

    new_names={'address': 'accession_number', "form": 'submission_type', 'putCall':'put_call', 'titleOfClass':'title_of_class',
               'nameOfIssuer':'name_of_issuer', 'cusip9':'cusip', 'shrsOrPrnAmt':'shares_bonds'}

    final_columns = [
        "cik",
        "cik_name",
        "rdate",
        "fdate",        
        "cusip",
        "name_of_issuer",       
        "value",
        "shares",
        "title_of_class",
        "shares_bonds",
        "put_call",
        "accession_number",
        "submission_type",
        "type",
        "file",
        'dsource',
        "entry_total",
        "value_total",
        "data_load"
    ]
    
    final_dtypes = {
            
            "cik": "Int64",
            'cik_name': str,
            "rdate": "datetime64[ns]",           
            "fdate": "datetime64[ns]",
            "cusip": str,
            "name_of_issuer": str,
            "value": "float64",
            "shares": "float64",
            "title_of_class": str,    
            "shares_bonds": str,
            "put_call": str,
            "accession_number": str,
            "submission_type": str,
            "type": str,
            "file": str,
            'dsource': str,
            "entry_total": "Int64",
            "value_total": "float64",
            "data_load": "datetime64[ns]"
        }
    path_list = list(csv_folder.glob('*.csv'))
    cutoff_date = '2014-01-01'
    
    for file_path in path_list:
        try:
            df = pl.read_csv(file_path, columns=columns, dtypes=pl_dtypes, parse_dates=True) 
            df = df.to_pandas().astype(pd_dtypes_validation)
            if df.empty or df.fdate.max() >= pd.to_datetime(cutoff_date,format='%Y-%m-%d'): continue
            df = (df.assign(value=df.value*1_000,
                            cik_name=None,
                            entry_total=None,
                            value_total=None,
                            file=file_path.as_posix(),
                            address=df.address.str.split('/').str[1].str.split('.').str[0],
                            data_load=pd.to_datetime('2023-01-01 00:00:00'))
                    .rename(columns=new_names)
                    .reindex(columns=final_columns)
                    .astype(final_dtypes))
            
            df.to_parquet(Path.joinpath(parq_folder, file_path.stem + f"-{df.head(1).fdate[0].strftime('%Y-%m-%d')}.parquet"), index=False)

        except Exception as e:
            print(f"An error occurred: {e}")
    # return df

In [ ]:
%%time
from pathlib import Path
clean_csv_0_2013 = Path(r"/Users/yo_macbook/Documents/app_data/dropbox_13f_files/processed_tables/TR_01_ALL_CSV_CLEANED")
clean_parq_0_2013 = Path(r'/Users/yo_macbook/Documents/app_data/dropbox_13f_files/processed_tables/TR_02_CLEANED_PARQ_0_2013')

test_parq = Path(r'/Users/yo_macbook/Documents/app_data/sec_apps_data/test_parq')

# data = clean_csv_to_parquet(clean_csv_0_2013,test_parq)
clean_csv_to_parquet(clean_csv_0_2013,clean_parq_0_2013)
# Wall time: 34min 27s

In [ ]:
data.head(3)
# data.info()
# data.value.sum()/1_000_000

## Here we take the cleaned `parquet` files and:
* Adding new column `data_load` with default value `'2023-01-01 00:00:00'`. It means this is the data that was manually dealt with and cleaned. It's the first baseline and a one-off task. 
* Filter data to only have filings where 'fdate' >= `2014-01-01`
* All the rest will come from the manually cleaned `csv` filings
* !!! **IMPORTANT** !!! this code multiplies manually cleaned `parquet` value by 1000

In [ ]:
%%time
from pathlib import Path
import pandas as pd
import polars as pl

def parq_from_2014(parq_clean_folder, parq_final_folder):
    """
    !!! IMPORTANT !!! this code multiplies manually cleaned csv value by 1000
    input: list of parquet file paths to multiply by 1_000 and add `data_load` column
    returns: enriched parquet
    """
#     columns = ['cik', 'cusip9','value', 'shares','rdate', 'fdate',\
#             'address', 'form', 'shrsOrPrnAmt', 'putCall', 'nameOfIssuer', 'titleOfClass', 'type', 'dsource']

#     pl_dtypes = {'cik' : pl.Int64, 'cusip9': str, 'value': pl.Float64, 'shares': pl.Float64, 'address': str,
#                  'form': str, 'shrsOrPrnAmt': str, 'putCall': str, 'nameOfIssuer': str, 'titleOfClass': str,
#                  'type': str, 'dsource': str}

#     pd_dtypes_validation = {'cusip9': str , 'titleOfClass': str, 'form': 'category',
#                 'putCall': 'category', 'shrsOrPrnAmt': 'category', 'value': 'float64',
#                 'shares': 'float64', 'type': 'category', 'nameOfIssuer': str,
#                 'cik' : 'int64', 'address': 'category',  'dsource': 'category'}

#     new_names={'address': 'accession_number', "form": 'submission_type', 'putCall':'put_call', 'titleOfClass':'title_of_class',
#                'nameOfIssuer':'name_of_issuer', 'cusip9':'cusip', 'shrsOrPrnAmt':'shares_bonds'}

    final_columns = [
        "cik",
        "cik_name",
        "rdate",
        "fdate",        
        "cusip",
        "name_of_issuer",       
        "value",
        "shares",
        "title_of_class",
        "shares_bonds",
        "put_call",
        "accession_number",
        "submission_type",
        "type",
        "file",
        'dsource',
        "entry_total",
        "value_total",
        "data_load"
    ]
    
    final_dtypes = {
            
            "cik": "Int64",
            'cik_name': str,
            "rdate": "datetime64[ns]",           
            "fdate": "datetime64[ns]",
            "cusip": str,
            "name_of_issuer": str,
            "value": "float64",
            "shares": "float64",
            "title_of_class": str,    
            "shares_bonds": str,
            "put_call": str,
            "accession_number": str,
            "submission_type": str,
            "type": str,
            "file": str,
            'dsource': str,
            "entry_total": "Int64",
            "value_total": "float64"
        }
    path_list = list(parq_clean_folder.glob('*.parquet'))
    cutoff_date = '2014-01-01'
    prior_2014 = Path(r'/Users/yo_macbook/Documents/app_data/sec_apps_data/prior_2014')
    
    for file_path in path_list:
        try:          
            df = pl.read_parquet(file_path) 
            df = df.to_pandas().astype(final_dtypes)
            if df.empty or df.fdate.max() < pd.to_datetime(cutoff_date,format='%Y-%m-%d'):
                df.to_parquet(Path.joinpath(prior_2014, 'prior_2014-'+file_path.name), index=False)
                continue               
            df = (df.assign(data_load=pd.to_datetime('2023-01-01 00:00:00'),
                            value=df.value*1_000)
                  .reindex(columns=final_columns))

            
            df.to_parquet(Path.joinpath(parq_final_folder, file_path.name), index=False)

        except Exception as e:
            print(f"An error occurred: {e}")


In [ ]:
%%time
TR_02_ALL_PARQ_SEC_APP = Path(r'/Users/yo_macbook/Documents/app_data/sec_apps_data/TR_02_ALL_PARQ_SEC_APP')
TR_02_FINAL_PARQ_2014_2023_02_17 = Path(r'/Users/yo_macbook/Documents/app_data/sec_apps_data/TR_02_FINAL_PARQ_2014_2023_02_17')

# test_parq = Path(r'/Users/yo_macbook/Documents/app_data/sec_apps_data/test_parq')

parq_from_2014(TR_02_ALL_PARQ_SEC_APP, TR_02_FINAL_PARQ_2014_2023_02_17)

In [ ]:
'prior_2014-'+file_path.name

## Rename `data_load` to `created_at` and add `updated_at`

In [ ]:

%%time
from pathlib import Path
import pandas as pd
import polars as pl

def add_updated_at(folder):    
    final_dtypes = {
            
            "cik": "Int64",
            'cik_name': str,
            "rdate": "datetime64[ns]",           
            "fdate": "datetime64[ns]",
            "cusip": str,
            "name_of_issuer": str,
            "value": "float64",
            "shares": "float64",
            "title_of_class": str,    
            "shares_bonds": str,
            "put_call": str,
            "accession_number": str,
            "submission_type": str,
            "type": str,
            "file": str,
            'dsource': str,
            "entry_total": "Int64",
            "value_total": "float64"
        }
    
    final_columns = [
        "cik",
        "cik_name",
        "rdate",
        "fdate",        
        "cusip",
        "name_of_issuer",       
        "value",
        "shares",
        "title_of_class",
        "shares_bonds",
        "put_call",
        "accession_number",
        "submission_type",
        "type",
        "file",
        'dsource',
        "entry_total",
        "value_total",
        "created_at",
        "updated_at"
    ]
    path_list = list(folder.glob('*.parquet'))

    for file_path in path_list:
        try:          
            df = pl.read_parquet(file_path) 
            df = df.to_pandas().astype(final_dtypes)           
            df = (df.assign(updated_at=pd.to_datetime('2023-01-01 00:00:00'))
                    .rename(columns={'data_load':'created_at'})
                    .reindex(columns=final_columns))

            
            df.to_parquet(file_path, index=False)

        except Exception as e:
            print(f"An error occurred: {e}")


In [ ]:
%%time
final_folder = Path(r'/Users/yo_macbook/Documents/app_data/TR_02_PARQ_ALL')
test = Path(r'/Users/yo_macbook/Documents/app_data/sec_apps_data/test_parq')

add_updated_at(final_folder)

## Add `data_load_run`

In [ ]:
%%time
from pathlib import Path
import pandas as pd
import polars as pl
from datetime import datetime

def add_data_load_run(folder):    
    final_dtypes = {
            
            "cik": "Int64",
            'cik_name': str,
            "rdate": "datetime64[ns]",           
            "fdate": "datetime64[ns]",
            "cusip": str,
            "name_of_issuer": str,
            "value": "float64",
            "shares": "float64",
            "title_of_class": str,    
            "shares_bonds": str,
            "put_call": str,
            "accession_number": str,
            "submission_type": str,
            "type": str,
            "file": str,
            'dsource': str,
            "entry_total": "Int64",
            "value_total": "float64",
            "created_at": "datetime64[ns]",
            "updated_at": "datetime64[ns]"
        }
    
    final_columns = [
        "cik",
        "cik_name",
        "rdate",
        "fdate",        
        "cusip",
        "name_of_issuer",       
        "value",
        "shares",
        "title_of_class",
        "shares_bonds",
        "put_call",
        "accession_number",
        "submission_type",
        "type",
        "file",
        'dsource',
        "entry_total",
        "value_total",
        "created_at",
        "updated_at",
        "data_load_run"
    ]
    path_list = list(folder.glob('*.parquet'))

    for file_path in path_list:
        try:          
            df = pl.read_parquet(file_path) 
            df = df.to_pandas().astype(final_dtypes)           
            df = (df.assign(data_load_run = "2023-01-01-01-01")
                    .reindex(columns=final_columns))
            
            df.to_parquet(file_path, index=False)

        except Exception as e:
            print(f"An error occurred: {e}")

In [ ]:
%%time
final_folder = Path(r'/Users/yo_macbook/Documents/app_data/TR_02_PARQ_ALL')
# test = Path(r'/Users/yo_macbook/Documents/app_data/sec_apps_data/test_parq')

add_data_load_run(final_folder)

In [ ]:
%%time

from pathlib import Path
import pandas as pd
import polars as pl
# from pandas_profiling import ProfileReport
from scipy import stats

selected_cik = [2230,3520,5272,7195,7789,9015,10742,14661,16972,18349,18748,19475,19617,21175,22657,24386,35442,35527,36066,36104,36644,36966,38777,39263,40417,40545,44365,45319,49205,50863,51762,51812,51964,52234,53417,59558,59951,60086,61227,67698,70858,71210,71259,72971,73124,80255,84616,89014,92230,93751,98758,102212,102909,105495,108572,200217,201772,216851,276101,310051,312348,313028,313807,314949,314957,314984,315014,315032,315038,315054,315066,315080,315157,315297,315498,316011,318989,320335,320376,351051,351173,351262,354204,356264,700529,704051,707179,712537,713676,714142,720672,723204,728083,728100,728618,732905,733020,740272,740913,741073,743127,750641,754811,757657,759944,762152,763212,763848,764068,764106,764112,764529,764532,765443,769317,769954,769963,775368,776867,778963,779519,788714,790354,790502,791191,791490,796848,799003,799004,801051,806097,807249,807985,808722,809339,809443,810265,810384,810386,810672,810716,811360,811454,813917,813933,814133,814375,816788,819535,820027,820123,820124,820289,820478,820743,821197,822581,823621,825293,829407,831001,831571,836372,837592,842782,842941,846222,846633,846788,846797,850401,850529,852743,854157,857508,859872,860486,860561,860580,860585,860643,860644,860645,860662,860748,860828,860857,861176,861177,861462,861787,862469,866361,866842,868491,869178,869179,869353,869367,872080,872163,872259,872573,872732,873630,874791,877134,877338,878228,881432,883511,883677,883782,883790,883803,883961,883965,884300,884314,884414,884423,884541,884546,884548,884566,884589,885062,885415,886982,887402,887777,887818,889232,891287,891478,893738,894205,894300,894309,895213,895421,897070,897378,897599,898358,898382,898399,898413,899211,900169,900529,900973,902219,902367,902464,902584,903064,903944,903947,903949,905567,905591,905608,906304,908195,909151,909661,911274,912938,914933,914976,915287,915325,916542,917579,918893,919079,919185,919192,919458,919489,919497,919530,919538,919859,920440,920441,921531,921669,922127,922439,922898,922940,923093,923116,923469,924166,924171,924181,926688,926833,926834,928047,928196,928566,928568,928633,930441,931097,932024,932974,933429,934639,934999,936698,936753,936936,936941,936944,937394,937522,937589,937615,937760,937886,938076,938206,938487,938582,938592,938759,939219,940445,941560,943719,944234,944804,945625,945631,947822,947996,948518,948669,949012,949509,949615,949623,949853,1000097,1000742,1002152,1002672,1002784,1004244,1005354,1005607,1005817,1006364,1006378,1006407,1006435,1007280,1007399,1007524,1008322,1008877,1008894,1008895,1008929,1008937,1009003,1009005,1009012,1009016,1009022,1009076,1009207,1009209,1009232,1009254,1009258,1009262,1010873,1010911,1011443,1011659,1013234,1013536,1013538,1013701,1014306,1014315,1014736,1014738,1015079,1015083,1015086,1015308,1016150,1016287,1016683,1016972,1017115,1017645,1017918,1018331,1018674,1018825,1019231,1020066,1020317,1020580,1020585,1020617,1020918,1021008,1021117,1021223,1021249,1021258,1021642,1021926,1023279,1024716,1025421,1026200,1026710,1027451,1027796,1027817,1029160,1030618,1030815,1031972,1032814,1033225,1033427,1033475,1033505,1033974,1033984,1034184,1034196,1034524,1034541,1034546,1034549,1034642,1034771,1034886,1035350,1035463,1035912,1036248,1036325,1037389,1037558,1037763,1037792,1038661,1039565,1039807,1040190,1040197,1040198,1040210,1040273,1040592,1040762,1041241,1041885,1042046,1044207,1044797,1044905,1044924,1044929,1044936,1046187,1047339,1048921,1049648,1049650,1050442,1050463,1050470,1051359,1052100,1053013,1053054,1053055,1054074,1054425,1054522,1054554,1054677,1055290,1055544,1055963,1055964,1055966,1056053,1056288,1056466,1056488,1056491,1056515,1056516,1056527,1056549,1056559,1056581,1056593,1056807,1056821,1056825,1056827,1056831,1056859,1056958,1056973,1057395,1057439,1058022,1058470,1058800,1059187,1061186,1061768,1062938,1065349,1065350,1066816,1067324,1067926,1067983,1068829,1070134,1071483,1072843,1074027,1074034,1074266,1074273,1076598,1077148,1077583,1078013,1078246,1078658,1078841,1079112,1079114,1079736,1079738,1079930,1080071,1080107,1080117,1080132,1080166,1080171,1080173,1080197,1080201,1080351,1080374,1080380,1080381,1080382,1080386,1080493,1080523,1080628,1080818,1081019,1081198,1082020,1082215,1082327,1082339,1082461,1082491,1082509,1082621,1082917,1083323,1083340,1084207,1084208,1084683,1085041,1085163,1085227,1085601,1085936,1086477,1086483,1086611,1086619,1086762,1086763,1088859,1088875,1088950,1089707,1089755,1089911,1089991,1090413,1091561,1091860,1091923,1092203,1092290,1092351,1092903,1093276,1093589,1094584,1094749,1095836,1096783,1097218,1097278,1097833,1100710,1101250,1102062,1102578,1102598,1103245,1103738,1103804,1103882,1103887,1104186,1104329,1104366,1105468,1105471,1105497,1105837,1105863,1105909,1106129,1106191,1106500,1106505,1106832,1107261,1107310,1108893,1108965,1108969,1109147,1110806,1113629,1114618,1114739,1114928,1115941,1116247,1125727,1125816,1129770,1133219,1134152,1140334,1140771,1142031,1142062,1158583,1389426,1398739,1469219]
TR_01_TEST_676_CIK_CSV_ORIGIN_CLEAN = Path(r"E:\app_data\dropbox_13f_files\processed_tables\TR_01_TEST_676_CIK_CSV_ORIGIN_CLEAN")
TR_01_TEST_676_CIK_CSV_CLEANED = Path(r"E:\app_data\dropbox_13f_files\processed_tables\TR_01_TEST_676_CIK_CSV_CLEANED")
both = Path(r"/Users/yo_macbook/Documents/app_data/dropbox_13f_files/processed_tables/TR_01_TEST_676_CIK_CSV_CLEANED_BOTH")

# both = Path(r"E:\app_data\dropbox_13f_files\processed_tables\TR_01_TEST_676_CIK_CSV_CLEANED_BOTH") # fanpc

columns = ['cik', 'cusip9','value', 'shares','rdate', 'fdate',\
            'address', 'form', 'shrsOrPrnAmt', 'putCall', 'nameOfIssuer', 'titleOfClass', 'type', 'dsource']

pl_dtypes = {'cusip8': str, 'cusip9': str , 'titleOfClass': str, 'form': str, 'putCall': str,
            'shrsOrPrnAmt': str, 'value': pl.Float64, 'shares': pl.Float64, 'type': str, 'nameOfIssuer': str,
            'cik' : pl.Int64, 'address': str,  'dsource': str}

pd_dtypes_validation = {'cusip9': str , 'titleOfClass': str, 'form': 'category',
            'putCall': 'category', 'shrsOrPrnAmt': 'category', 'value': 'float64',
            'shares': 'float64', 'type': 'category', 'nameOfIssuer': str,
            'cik' : 'int64', 'address': 'category',  'dsource': 'category'}


cleaned = [file for file in TR_01_TEST_676_CIK_CSV_CLEANED.glob("*.csv")]
origin_clean = [file for file in TR_01_TEST_676_CIK_CSV_ORIGIN_CLEAN.glob("*.csv")]
both_clean = [file for file in both.glob("*.csv")]



cik_dfs = []
for index, cik in enumerate(selected_cik[:10]):
    file_dfs = []
    for file in set(both_clean):
    # for file in set(cleaned + origin_clean):
        if file.name.split("-")[0] == str(cik):
            try:
                df = pl.read_csv(file, columns=columns, dtypes=pl_dtypes, parse_dates=True) 
                
                df = df.to_pandas().astype(pd_dtypes_validation)
                if df.empty: continue

                df = (df.assign(row_value_zscore = stats.zscore(df.value),
                                file_value_sum = df.value.sum(),
                                file_value_max = df.value.max(),
                                file_value_min = df.value.min(),
                                file_record_count = df.shape[0],
                                quarter=df.rdate.dt.to_period(freq="Q").astype(str)))
                df_short = df[['cik', 'cusip9', 'rdate', 'fdate',
                               'address', 'file_value_sum', 'file_value_max',
                               'file_value_min', 'file_record_count', 'quarter']].head(1)
               

                file_dfs.append(df_short)
                df = None
            except Exception as e:
                print(f"Problem reading file... {file.name}")
                print(e)
                # os.system(f"code "+ file.as_posix())
                # i = input("did you correct the file?")
                # if i == "yes":
                #     print(f"file: {file} is finished")
                #     continue

                # else: 
                #     sys.exit()
        
    cik_df = pd.concat(file_dfs)
    file_dfs = None

    cik_dfs.append(cik_df)
big_df = pd.concat(cik_dfs)
    # df = pl.concat(dfs).to_pandas().astype(pd_dtypes_validation)
    # df = df.to_pandas().astype(pd_dtypes_validation)

In [ ]:
import altair as alt
from altair import datum
alt.data_transformers.disable_max_rows()

input_dropdown = alt.binding_select(options=selected_cik, name='cik ')
cik_selection = alt.selection_single(fields=['cik'], bind=input_dropdown)
brush = alt.selection(type='multi')

scatter = alt.Chart(big_df).mark_point().transform_calculate(value_billion="datum.value / 1000000000")\
    .encode(
    alt.X('quarter:O'),
    alt.Y('file_value_sum:Q'),
    
    # alt.Row('cik'),
    tooltip=['quarter:O','address:N', 
             alt.Tooltip('file_value_sum:Q', format="$.3s") ,
             alt.Tooltip('file_value_sum:Q', format="$~s"),
             alt.Tooltip('fdate:T')
            ]
    # alt.Tooltip(['quarter:O','accession_number:N', 'value:O']),
    
).add_selection(cik_selection
).transform_filter(cik_selection
).add_selection(brush
).properties(width=800, height=200)

ranked_text = alt.Chart(big_df).mark_text().encode(
    y=alt.Y('row_number:O',axis=None)
).transform_window(
    row_number='row_number()'
).transform_filter(
    brush
).transform_window(
    rank='rank(row_number)'
).transform_filter(
    alt.datum.rank<5
)

# Data Tables
address = ranked_text.encode(text='address:N').properties(title='address')
quarter = ranked_text.encode(text='quarter:O').properties(title='quarter')
value = ranked_text.encode(text='file_value_sum:Q').properties(title='value')
text = alt.hconcat(address, quarter, value) # Combine data tables

alt.vconcat(
    scatter,
    text
).resolve_legend(
    color="independent"
).configure_view(
    strokeWidth=0
)

In [ ]:
import altair as alt
import pandas as pd
import panel as pn

pn.extension("vega", sizing_mode="stretch_width")


theme = pn.state.session_args.get("theme", [b"default"])[0].decode()
if theme == "dark":
    alt.themes.enable("dark")
else:
    alt.themes.enable("default")

    
    
import altair as alt
from altair import datum
alt.data_transformers.disable_max_rows()

input_dropdown = alt.binding_select(options=selected_cik, name='cik ')
cik_selection = alt.selection_single(fields=['cik'], bind=input_dropdown, name='cik')
brush = alt.selection(type='interval', name='brush')

scatter = alt.Chart(big_df).mark_point().transform_calculate(value_billion="datum.value / 1000000000")\
    .encode(
    alt.X('quarter:O'),
    alt.Y('file_value_sum:Q'),
    
    # alt.Row('cik'),
    tooltip=['quarter:O','address:N', 
             alt.Tooltip('file_value_sum:Q', format="$.3s") ,
             alt.Tooltip('file_value_sum:Q', format="$~s"),
             alt.Tooltip('fdate:T')
            ]
    # alt.Tooltip(['quarter:O','accession_number:N', 'value:O']),
    
).add_selection(cik_selection
).transform_filter(cik_selection
).add_selection(brush
).properties(width=800, height=200)

vega_pane = pn.pane.Vega(scatter, debounce=10, height=600)

@pn.depends(vega_pane.selection.param.brush, vega_pane.selection.param.cik)
def filtered_table(selection, selection2):
    return selection
    if not selection:
        return "## No selection"
    # query = " & ".join(
    #     f'{crange[0]:.3f} <= `{col}` <= {crange[1]:.3f}'
    #     for col, crange in selection.items()
    # )
    # return pn.pane.DataFrame(data.query(query))


# Data Tables


table = pn.Row(filtered_table, height=300, scroll=True)
component = pn.Column(vega_pane, table).servable()
component

# template = pn.template.FastListTemplate(
#     site="Awesome Panel",
#     title="Panel supports Vega and Altair Selections",
#     accent="#F08080",
#     main=[component],
# ).servable()

In [ ]:
both = Path(r"E:\app_data\dropbox_13f_files\processed_tables\TR_01_TEST_676_CIK_CSV_CLEANED_BOTH")
l = ['105495-0001193125-04-000834.csv', '105495-0001193125-03-057541.csv','105495-0001193125-03-018135.csv',
    '105495-0000927016-03-001711.csv', '105495-0000927016-03-000073.csv', '105495-0000927016-02-004805.csv',
     '105495-0000927016-02-003580.csv', '105495-0000927016-02-002002.csv', '105495-0000927016-02-000170.csv',
     '105495-0000927016-01-503075.csv', '105495-0000927016-01-501869.csv']

for file in set(both_clean):
    if file.name in l:
        try:
            print(Path.joinpath(both, file.name))
            df = pl.read_csv(file, columns=columns, dtypes=pl_dtypes, parse_dates=True) 
            df = df.to_pandas().astype(pd_dtypes_validation)
            df = (df.assign(value=df.value/1000))
            df.to_csv(Path.joinpath(both, file.name), index=False)
        except:
            print("error")

# big_df

In [ ]:
df.address.iloc[0]
# df.value.sum()
file.name
Path.joinpath(both_clean, file.name)

In [ ]:
%%time
# more_than_one_match = {'file': [], 'bad_cusip': [], 'new_cusip': []}
# cik_big_dfs[:3]
df.head(),
df.shape

# pandas formatting, format
format_dict = {col_name: '${:,.0f}' for col_name in cik_df.select_dtypes(float).columns}
# override the percentage column
format_dict['row_value_zscore'] = '{:.2f}'
format_dict['shares'] = '{:.0f}'

# cik_df.head().style.format(format_dict)
# cik_df.sort_values(by=['value'], ascending=False).head(3).style.format(format_dict)

# summary statistics
# from skimpy import skim
# skim(cik_df)

#####################
# import sweetviz as sv

# my_report = sv.analyze(cik_df[['file_value_sum','quarter']],target_feat='file_value_sum')
# # # my_report.show_html() # Default arguments will generate to "SWEETVIZ_REPORT.html"
# my_report.show_notebook(layout='vertical') # Default arguments will generate to "SWEETVIZ_REPORT.html"

###########################
# import data_describe as dd
# dd.data_summary(cik_df[['value','file_value_sum','quarter']]) # , as_percentage=True
# dd.data_heatmap(cik_df[['value','file_value_sum','quarter']])
# from IPython.display import display
# display is used to show plots from inside a loop
# for col in ['file_value_sum','quarter']:
#     display(dd.distribution(cik_df[['file_value_sum','quarter']], plot_all=False).plot_distribution(col))
#####
# dist = dd.distribution(df)
# dist
# dist.plot_distribution("value", mode='hist')
# dist.skew_value
    
# dd.correlation_matrix(cik_df)
# dd.correlation_matrix(cik_df, cluster=True)
#####
# dd.scatter_plots(cik_df[['file_value_sum','quarter']], mode='diagnostic', threshold={'Outlying': 0.5})
# dd.scatter_plots(cik_df[['file_value_sum','quarter']], mode='all')
#####
# c = dd.cluster(df)
# c.show()
##############
from pandas_profiling import ProfileReport
profile = ProfileReport(cik_df[['file_value_sum']], title="Pandas Profiling Report")
profile.to_widgets()

# cik_df.head()

In [ ]:
cik_df.query("file_value_sum ==9467399625.0")
big_df.head()

In [ ]:

brush = alt.selection(type='interval')
points = alt.Chart(big_df[['cik', 'address', 'file_value_sum', 'quarter']]).mark_point().encode(
    x='quarter:O',
    y='file_value_sum:Q',
    
    color=alt.condition(brush, 'quarter', alt.value('lightgray'))
).add_selection(brush)

bars = alt.Chart(big_df[['cik', 'address', 'file_value_sum', 'quarter']]).mark_bar().encode(
    y='address:N',
    # color='fdate:T',
    x='file_value_sum'
).transform_filter(brush)
# points
points & bars

In [ ]:
import pandera as pa
value_10B_schema = pa.DataFrameSchema({
    "value": pa.Column(float, pa.Check(lambda s: s <= 10000000), required=True, nullable=False)
})

try:
    value_10B_schema.validate(cik_df) , #lazy=True

except pa.errors.SchemaErrors as e:
    failure_cases = e.failure_cases

In [ ]:
%%time
from pathlib import Path

for index, row in big_bad_df.iterrows():
    # if (row.failure_case not in processed_cusip) and (row.failure_case not in processed_no_match):
    file_name = Path(row.df_file).name.split(".")[0].split("-", maxsplit=1)[-1]
    print(row.df_file)
    bad_cik = row.df_cik
    bad_cusip = row.failure_case
    bad_rdate = row.df_rdate.date()

    match = cusip_676.query(f'address.str.contains("{bad_cik}/{file_name}") &\
                  cusip.notna() &\
                  cusip.str.contains(@bad_cusip)')['cusip'].drop_duplicates().squeeze() # &\
    
 # & rdate == @bad_rdate'

    if not any(match):
        print("no match")
        processed_no_match.append(row.failure_case)
        continue
    elif any(match) and not isinstance(match, pd.core.series.Series):       
        print(f"bad cusip: {bad_cusip} --- new cusip: {match}")
        df = pl.read_csv(row.df_file, columns=columns, dtypes=pl_dtypes, parse_dates=True)
        df = df.to_pandas().astype(pd_dtypes_validation)
        df = (df.assign(cusip9=df.cusip9.str.replace(bad_cusip, match)))
        df.to_csv(row.df_file, index=False)
        processed_cusip.append(row.failure_case)
        
    elif isinstance(match, pd.core.series.Series):
        print("more than one match")
        more_than_one_match['file'].append(row.df_file)
        more_than_one_match['bad_cusip'].append(bad_cusip)
        more_than_one_match['new_cusip'].append(match.to_list())




In [ ]:
isinstance(match, pd.core.series.Series)
# isinstance([], list)
# pd.dtype(match)
match.to_list()

In [ ]:
# df.query("cusip9== '690733209'")
len(processed_cusip)
# processed_cusip[:3]
# match.head(1).squeeze()
# bad_cusip # '565849'

len(match)


In [ ]:
# directory = Path(r"E:\app_data\dropbox_13f_files\processed_tables")
# cusip_676 = pd.read_parquet(Path.joinpath(directory, "value_total_676.parquet")) 
# cusip_676.head()
# cusip_676 = big_df_cusip_676.drop_duplicates(subset=['cusip'])

##
# md_dir = Path(r"E:\app_data\sec_apps_data\speed_test\filings_13f_full\master_data\cusip")
# cusip_merged_final = pd.read_parquet(Path.joinpath(md_dir, "cusip_merged_final.parquet")) 
# cusip_merged_final.head()
##
bad_cik = 3520
bad_cusip = "0733209"
cusip_676.query(f'address.str.contains("{bad_cik}/") &\
                  cusip.notna() &\
                  cusip.str.contains("{bad_cusip}") &\
                  rdate == "1999-12-31"')['cusip']
# "0733209" in (cusip_676.cusip.values or cusip_merged_final.cusip.values)
# "088033G10" in (cusip_676.cusip.values or cusip_merged_final.cusip.values)
# for cusip in cusip_676.cusip:
#     if "G03910109" == cusip:
#         print(cusip)
# for cusip in cusip_merged_final.cusip:
#     if "G03910109" == cusip:
#         print(cusip)
# cusip_676.info()

In [ ]:
# md_02_ftd_cusip_ticker.parquet
# cusip_merged_final.query("cusip.str.contains('45920010')")
cusip_676.query('cusip.notna() & cusip.str.contains("141705SH")')
# cusip_676.query('cusip.notna() & cusip.str.contains("45920010")')

# df.query('index.isin([24, 80])').head(7)

# 0     26       1957109
# 1    183     718154107
# 2   1369     297659104
# 3   4145     362320103
# 4   4358      16962105
# 5   4373      67543101
# 6   4383      90078109

In [ ]:
cusip_676.query('cusip.notna() & cusip.str.contains("E")')

In [ ]:
basic_types_schema = pa.DataFrameSchema({
    "cik": pa.Column('int64'),
    "cusip8": pa.Column(str),
    "cusip9": pa.Column(str, pa.Check(lambda s: s.str.len() == 9)),
    "rdate": pa.Column("datetime64"),
    "fdate": pa.Column("datetime64"),
    "value": pa.Column("float64"),
    "shares": pa.Column("float64")
    
    })

In [ ]:
basic_types_schema.validate(df[columns])

In [ ]:
basic_types_schema = pa.DataFrameSchema({
    "cusip9": pa.Column(str, pa.Check(lambda s: ~s.str.contains("\+|\-")))
                        })
    # "cik": pa.Column('int64'),
    # "cusip8": pa.Column(str),
    # "rdate": pa.Column("datetime64"),
    # "fdate": pa.Column("datetime64"),
    # "value": pa.Column("float64"),
    # "shares": pa.Column("float64") 

In [ ]:
%%time
# Pandera also allows validating value ranges for numerical columns
# value_range_schema = pa.DataFrameSchema({
#     "LotArea": pa.Column(int, pa.Check(lambda s: s <= 1000000), nullable=False),
#     "YearBuilt": pa.Column(int, pa.Check.in_range(1800, 2022)),
# })

value_range_schema = pa.DataFrameSchema({
    "cik": pa.Column('int64'),
    "cusip8": pa.Column(str, pa.Check(lambda s: s.str.len() >= 8), pa.Check(lambda s: s.str.len() <= 9)),
    "cusip9": pa.Column(str, pa.Check(lambda s: s.str.len() >= 9), pa.Check(lambda s: s.str.len() <= 9)),

    # "cusip9": pa.Column(str, pa.Check(lambda s: ~s.str.contains("\+|\-"))),
    # "rdate": pa.Column("datetime64"),
    # "fdate": pa.Column("datetime64"),
    # "value": pa.Column("int64", pa.Check(lambda s: s <= 1000000), nullable=False)
                        })
%time value_range_schema.validate(df[columns])



In [ ]:
filter = df.cusip9.str.contains('\+')
df.loc[filter]

In [ ]:
df.cusip9.sort_values(ascending=True)

In [ ]:
files = filings_676_txt.rglob("*1006435/*.txt")
total_n_filings = len(list(filings_676_txt.rglob("*1006435/*.txt")))
total_n_filings

In [ ]:
list(filings_676_txt.rglob("*.txt"))[0]

In [ ]:
%%time
import re
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime

# Define the regular expression to extrtact `cusip` values
exp_cusip = r"\b[A-Za-z0-9]{9}(?<![A-Za-z]{9})\b"

# Compile the regular expression
pattern_cusip = re.compile(exp_cusip)

# Open the text file
filings_676_txt = Path(r"E:\app_data\dropbox_13f_files\processed_tables\TR_00_TEST_676_CIK_TXT")
files = filings_676_txt.rglob("*.txt")  # -03-000131
total_n_filings = len(list(filings_676_txt.rglob("*.txt")))
big_df = pd.DataFrame()
###
df_processed = pd.read_parquet("df_processed.parquet")
filings_processed = [tuple(x) for x in df_processed.itertuples(index=False, name=None)]
###
df_to_correct.to_parquet("df_to_correct.parquet", index=False)
filings_to_correct = [tuple(x) for x in df_to_correct.itertuples(index=False, name=None)]
filings_other = []

for file in files:
    if file.name not in (filings_processed or filings_manually_corrected):
        for file in files:
            text = file.read_text()
            # print(file)    
            xml_pattern = r'<?xml'
            match_xml = re.search(xml_pattern, text, re.DOTALL | re.IGNORECASE)
            if match_xml: continue

            data_dict = dict()
            data_dict['rdate'] = datetime.strptime(
                re.compile(r"(?<=CONFORMED PERIOD OF REPORT:).*")
                .search(text)
                .group(0)
                .strip(),
                "%Y%m%d",
            ).date()

            data_dict['fdate'] = datetime.strptime(
                re.compile(r"(?<=FILED AS OF DATE:).*")
                .search(text)
                .group(0)
                .strip(),
                "%Y%m%d",
            ).date()

            accession = (
                re.compile(r"(?<=ACCESSION NUMBER:).*")
                .search(text)
                .group(0)
                .strip()+".txt"
            )
            cik = (
                re.compile(r"(?<=CENTRAL INDEX KEY:).*")
                .search(text)
                .group(0)
                .strip().lstrip('0')
                )
            data_dict['address']  = '/'.join([cik, accession])

            ## !!! todo !!! problem with cases where there is no lines for entry or value total at all. code break at the missing .group(0)

            entry_total = re.compile(r"(Entry Total|Total Entry(:)?).*", flags = re.M|re.I).search(text)
            if entry_total:
                entry_total = entry_total.group(0).strip()
                entry_total = re.sub(r'\D', '', entry_total)
                if entry_total:
                    entry_total = int(entry_total)
                else:
                    entry_total = np.nan
            else:
                entry_total = np.nan
            data_dict['entry_total'] = entry_total

            #######----

            value_total = re.compile("(Value Total|Total Value(:)?).*", flags = re.M|re.I).search(text)
            if value_total:
                value_total = value_total.group(0).strip()
                value_total = re.sub(r'\D', '', value_total)
                if value_total:
                    value_total = int(value_total)
                else:
                    value_total = np.nan

            else:
                value_total = np.nan
            data_dict['value_total'] = value_total


            header_df = pd.DataFrame.from_dict([data_dict]).astype({'entry_total': 'Int64', 'value_total': 'Int64'})       
            if header_df.value_total.isna().any() or header_df.entry_total.isna().any() : 
                # print(header_df.head())
                filings_to_correct.append((file.name, data_dict['entry_total'], data_dict['value_total']))
            else: 
                filings_processed.append((file.name, data_dict['entry_total'], data_dict['value_total']))
             
            filings_other.append((file.name, data_dict['entry_total'], data_dict['value_total']))


            matches = pattern_cusip.finditer(text)
                # Create a list of the matched strings
            matched_strings = [match.group(0) for match in matches]
            matched_strings = set(matched_strings)
            matched_strings = set([s for s in matched_strings if not re.search(r'([^\d]{3,})', s)])

            df = pd.DataFrame(matched_strings, columns=['cusip'])
            # else:
            #     df = pd.DataFrame("no_cusip", columns=['cusip'])

            df = pd.concat([header_df, df], axis=1).ffill()
            big_df = pd.concat([big_df, df])

            # Filter out the strings that contain only non-digit characters
            # filtered_strings = set([s for s in matched_strings if any(c.isdigit() for c in s) and not all(c.isalpha() for c in s)])

# df_to_correct = pd.DataFrame(filings_to_correct, columns=['file', 'entry_total', 'value_total'])
# df_processed = pd.DataFrame(filings_processed, columns=['file', 'entry_total', 'value_total'])

In [ ]:

# total_n_filings # 67629
# len(filings_processed) # 32178
# filings_processed[-50:-1]
# len(filings_manually_corrected)
# len(filings_to_correct) # 8906
# filings_to_correct[0]
# filings_other[:2]


In [ ]:
# big_df.query("value_total.isna() & entry_total.notna()").drop_duplicates(subset=["address"]).shape
# # big_df.head()
# cusip_from_txt = (big_df.astype({'rdate':'datetime64', 'fdate':'datetime64'})
#                 .drop_duplicates(subset=["cusip"]))

cusip_from_txt.head()
# cusip_from_txt.info()

In [ ]:
# big_df = big_df.drop_duplicates() # before  # (15354045, 6)
# pd_dtypes = {'cusip': 'category' , 'value_total': 'Int64', 'entry_total': 'Int64', 'address': 'category',
#            'rdate': 'datetime64', 'fdate': 'datetime64'}

# directory = Path(r"E:\app_data\dropbox_13f_files\processed_tables")
big_df = big_df.astype(pd_dtypes)
big_df.info()
# big_df.to_parquet(Path.joinpath(directory, 'value_total_676.parquet'), index=False)

### Checking files that need to be corrected (no value or entry or both)

In [ ]:
###### big_df2 = pd.read_parquet(Path.joinpath(directory, "value_total_676.parquet")) 
## check '21175/0000021175-02-000013.txt' 
dont_have_value_in_txt = ['1026710', '1011659', '1041885']
# casaes with total entries, but no total values
fixed_value_cik = ('1008895' , '1002152', '1006435', '1011659', '1026710', '1033427',
                   '1033505' , '1035463', '1037389', '1040197', '1040273', '1041885', '1053013',
                   '1056516', '1077583', '1079112' , '1082339', '1082621', '1082917', '1085041',
                   '1085227', '1085936', '1088950', '1105468', '21175', '320335', '36066', '40545',
                   '741073', '754811', '790354', '861177', '866361', '869367', '872732', '877134',
                   '884566', '889232', '902584', '919079', '92230', '928047', '936944', '938076',
                   '939219', '947822')
cik = '947822'
####
(big_df2.query(f"value_total.isna() & \
                entry_total.notna() & \
                address.str.contains('|'.join(@fixed_value_cik))==False  ") 
         .drop_duplicates(subset=['address']))

# print(big_df2.query(f"value_total.isna() & \
#                 entry_total.notna() & \
#                 address.str.contains('|'.join(@fixed_value_cik))==False  ") 
#          .drop_duplicates(subset=['address'])['address'])

# l = (big_df2.query(f"value_total.isna() & \
#                 entry_total.notna() & \
#                 address.str.contains('|'.join(@fixed_value_cik))==False & address.str.contains('{cik}')  ") 
#          .drop_duplicates(subset=['address'])['address']).to_list()


# df.query("cusip == '066050105'")
# value_total
# matched_strings
# [s for s in matched_strings if not re.search(r'([^\d]{3,})', s).group(0)]
# [s for s in matched_strings if not re.search(r'([^\d]{3,})', s)]

In [ ]:
files = filings_676_txt.rglob(f"{cik}/*.txt")  # -03-000131

# l = ('1008895/0001008895-06-000002.txt',
#  '1008895/0001008895-06-000009.txt',
#  '1008895/0001008895-06-000011.txt')


for file in files:
    if f"{cik}/"+file.name in l:
        text = file.read_text()
######################################  Value Total is on a wrong line. Below the correct one  ###########
############ itendify the correct number/amount (current version only works if thousand separator is present) ##
##### adds correct amount in a correct place. After "Value Total"   ###################

        # select text 200 tokens befire and 200 after the "Entry Total" and work with it
#         pattern = re.compile(r"(?:^.{1,200})(?:Entry Total:?)(?:.{1,400}$)", flags=re.M|re.DOTALL)
#         working_text = (pattern.findall(text)[0])
#         # print(working_text, sep="\n")
      
#         lines = working_text.splitlines()
#         for index, line in enumerate(lines):
#             pattern_v_total = re.compile(r"((Value Total|Total Value)\s*(:?))", flags=re.I)
#             match_v_total = pattern_v_total.search(line)
#             if match_v_total:
#                 pattern_v_total_amnt = re.compile(r"((Value Total|Total Value)\s*(:?).*\d+)", flags=re.I)
#                 match_v_total_amnt = pattern_v_total_amnt.search(line)
#                 if match_v_total_amnt:
#                     continue
#                 else:
#                     pattern_next_line_amnt = re.compile(r"(.*(\d+))", flags=re.I)
#                     match_next_line_amnt = pattern_next_line_amnt.search(lines[index+1])
#                     if match_next_line_amnt:
#                         updated_line = match_v_total.group()+match_next_line_amnt.group() 
#                         updated_text = re.sub(pattern_v_total,updated_line, text)
#                         # print(updated_text[3500:6000], "#" * 10, sep="\n")
#                         file.write_text(updated_text)       

            
######################################  Value Total is on a wrong line. Above the correct one  ###########
############ itendifying the correct number/amount (current version only works if thousand separator is present) ##
##### adds correct amount in a correct place. After "Value Total"   ###################
##### also uses Path.read_text() and write_text() instead of `open with`
#         start_str = "Entry Total"
#         end_str = "Value Total"
#         match = re.search(r"{}(.*){}".format(start_str, end_str), text, flags=re.DOTALL)
#         if match:
#             value = match.group(1)
#             value = re.findall(r'\b\$?\d{1,3}(?:,\d{3})*(?:\.\d+)?\b', value)[-1]
#             value = re.sub(r'\.\d+', '', value)  
#             value = re.sub(r'\D', '', value)

#         value_total_old = re.compile(r"^(Form 13F Information Table Value Total).*?$", flags=re.M)
#         sub_match = re.search(value_total_old, match.group())
#         new_text = re.sub(value_total_old, fr"Form 13F Information Table Value Total: {value} ", text)
#         print(new_text[3500:6000], "#" * 10, sep="\n")
#         file.write_text(new_text)

#  
############################# swap multiple lines in text for one  #################################
#         entry_total = re.compile(r"(\nForm 13F Information Table Value Total:\n\n     \$)", re.IGNORECASE).finditer(text)
#         matches =  [match.group(0) for match in entry_total]
#         # print(matches)
#         if matches:  
#             updated_text = text.replace(r"""Form 13F Information Table Value Total:

#      $""", "Form 13F Information Table Value Total $")

#             print(updated_text[3500:5000])

#               #### Write the file out again
#         with open(file, 'w') as file:
#             file.write(updated_text)     

#############################################################################
#         entry_total = re.compile(r"(Form 13F Information Table Value Total:\n\$).*").finditer(text)
#         matches =  [match.group(0) for match in entry_total]
#         matches =  [match.replace("Form 13F Information Table Value Total:\n$", "") for match in matches]
#         # print(matches)
#         if matches:  
#             # updated_text = text.replace("Value (x $1000) Total", "(x $1000) Value Total", 1)
#             # updated_text = text.replace("Table Value $", "Table Value Total $")
#             updated_text = re.sub(r"(Form 13F Information Table Value Total:\n\$).*",\
#                                   f"Form 13F Information Table Value Total: {matches[0]}", text)
        
        
        
        
################################# Swap 2nd "Entry Total" for "Value Total"  #########################################        
#         # code to find the 2nd occurence of the substring in the text and 
#         # substitute only this 2nd ocurrence with something else 

#         entry_total = re.compile(r"(Entry Total)", re.IGNORECASE).finditer(text)
#         matches =  [match.group(0) for match in entry_total]
#         print(matches)
#         if len(matches) > 1:  
#             where = [m.start() for m in re.finditer("Entry Total", text)][2-1]
#             before = text[:where]
#             after = text[where:]
#             after = after.replace("Entry Total", "Value Total", 1)
#             updated_text = before + after
#             # print(updated_text[2000:3500])
            
# # #             # ### Write the file out again
#             file.write_text(updated_text)
    
# #             with open(file, 'w') as file:
# #                 file.write(updated_text)

            
            
##########################################################################
#         # code to find the 2nd occurence of the substring in the text and 
#         # substitute only this 2nd ocurrence with something else 
        
#         entry_total = re.compile(r"(Entry Total|Value Total)").finditer(text)
#         matches =  [match.group(0) for match in entry_total]
#         print(matches)
# #         if len(matches) > 1:  
#             where = [m.start() for m in re.finditer("Entry Total", text)][2-1]
#             before = text[:where]
#             after = text[where:]
#             after = after.replace("Entry Total", "Value Total", 1)
#             updated_text = before + after
#             # print(updated_text[2500:3500])
            
#             ### Write the file out again
#             with open(file, 'w') as file:
#                 file.write(updated_text)
        
###################################################################### Value \(x $1000\) Total
#         entry_total = re.compile(r"(Form 13F Information Table Value Total:\n\$).*").finditer(text)
#         matches =  [match.group(0) for match in entry_total]
#         matches =  [match.replace("Form 13F Information Table Value Total:\n$", "") for match in matches]
#         # print(matches)
#         if matches:  
#             # updated_text = text.replace("Value (x $1000) Total", "(x $1000) Value Total", 1)
#             # updated_text = text.replace("Table Value $", "Table Value Total $")
#             updated_text = re.sub(r"(Form 13F Information Table Value Total:\n\$).*",\
#                                   f"Form 13F Information Table Value Total: {matches[0]}", text)

#             # print(updated_text[2500:3000])
#           # #Write the file out again
#         with open(file, 'w') as file:
#             file.write(updated_text)
######################### swap of regex pattern for a text string "TotaL Value" for "Value Total" ##########################
#         entry_total = re.compile(r"(Total Value)", re.IGNORECASE).finditer(text)
#         matches =  [match.group(0) for match in entry_total]
#         # print(matches)
#         if matches:  
#             updated_text = re.sub(matches[0], "Value Total", text, flags=re.IGNORECASE)
#             # print(updated_text[1000:4000])

#         #  # Write the file out again
#         with open(file, 'w') as file:
#             file.write(updated_text)
            
################################# Simple Swap: "Values Total" for "Value Total"  ########################
#         entry_total = re.compile(r"(Value total)", re.IGNORECASE).finditer(text)
#         matches =  [match.group(0) for match in entry_total]
#         print(matches)
#         if matches:  
#             updated_text = re.sub(matches[0], "Value Total", text, flags=re.IGNORECASE)
#             # print(updated_text[2000:4000])

#          # Write the file out again
#         file.write_text(updated_text)

    
    

In [ ]:
text

In [ ]:
import pandas as pd

# Specify the delimiter, which in this case appears to be whitespace
delimiter = '\s+'
file = Path(r"C:\users\yo_fanpc\Desktop\filing_copy.txt")
# Read the file and create a pandas DataFrame
# df = pd.read_csv(file, delimiter=delimiter, engine='python', header=None,  index_col=False).astype({0:str, 1:})

# # Select the first 6 columns
# # df = df[df.columns[:6]]

# # display the dataframe
# df.head()
# df.



# widths = [33, 17, 9, 9, 9, 10]
# widths = [28, 7, 9, 16, 12, 4]
# widths = [33, 17, 9, 9, 9, 3]
# widths = [33, 6, 11, 14, 14, 3]
# widths = [32, 10, 16, 9, 9, 4]
widths = [31, 17, 9, 9, 9, 4]

columns = ['issuer', 'title', 'cusip', 'value', 'shares', 'shrsprn']
df = pd.read_fwf(file, widths=widths, names=columns,engine='python', header=None,  index_col=False, dtype=({'cusip':str})).ffill()

df.to_csv(Path.joinpath(file.parent, 'filing_copy.csv'), index=False)


In [ ]:
df.tail(300)

In [ ]:
d = pd.read_csv(Path.joinpath(file.parent, 'filing_copy.csv'))

In [ ]:
d.head(20)

In [ ]:
txt = """
AB,CD,EF, JJ
foo,20160101,a,23
foo,20160102,a,34
foo,20160103,a,56
"""
lambda txt: len(txt) <= 10
# data = pl.read_csv(io.StringIO(txt))
# data = data.

In [ ]:
# different pandas dtypes

dtypes =    {
        "ID": str,
        "accessionNumber": str,
        "cikManager": "Int64",
        "periodOfReport": "datetime64[ns]",
        "report_Quarter": "Int64",
        "report_Year": "Int64",
        "submissionType": str,
        "isAmendment": bool,
        "amendmentType": str,
        "filedAsOfDate": "datetime64[ns]",
        "entryTotal": "Int64",
        "valueTotal": "float64",
        "cusip": str,
        "nameOfIssuer": str,
        "titleOfClass": str,
        "sharesValue": "float64",
        "sharesHeldAtEndOfQtr": "Int64",
        "securityType": str,
        "putCall": str,
        "xml_flag": str,
        "created_at": "datetime64[ns]",
        "edgar_path": str,
    }